In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
#6.3.2 应用简化版SMO算法处理小规模数据集

In [3]:
def loadDataSet(fileName):
    dataMat,labelMat = [],[]
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [4]:
def selectJrand(i,m):
    j = i
    while j == i:
        j = int(np.random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
    return aj

In [5]:
data_path = Path('D:\\python_algorithm\\machinelearinginaction\\《机器学习实战》Python3代码\\Ch06')
dataArr,labelArr = loadDataSet(data_path / 'testSet.txt')

In [6]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()    #行向量转为列向量
    b = 0
    m,n = np.shape(dataMatrix)
    
    alphas = np.mat(np.zeros((m,1)))
    iter = 0
    
    while iter < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas,labelMat).T * \
                        (dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])
            
            if ((labelMat[i] * Ei) < -toler and alphas[i] < C) or \
                ((labelMat[i] * Ei) > toler and alphas[i] > 0):
                j = selectJrand(i,m)                                 #随机选择另外一个alpha
                
                fXj = float(np.multiply(alphas,labelMat).T * \
                        (dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                if labelMat[i] != labelMat[j]:
                    L = max(0,alphas[j] - alphas[i])
                    H = min(C,C+alphas[j] - alphas[i])
                else:
                    L = max(0,alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                    
                if L == H:
                    #print('L == H')
                    continue
                
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - \
                      dataMatrix[i,:] * dataMatrix[i,:].T - \
                      dataMatrix[j,:] * dataMatrix[j,:].T
                
                if eta >= 0:
                    #print('eta >= 0')
                    continue
                
                alphas[j] -= labelMat[j] * (Ei - Ej)/eta                      #求另外一个alpha的值
                alphas[j] = clipAlpha(alphas[j], H, L)
                
                if abs(alphas[j] - alphaJold) < 0.00001:
                    #print('j not moving enough')
                    continue
                
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * \
                    dataMatrix[i,:] * dataMatrix[i,:].T - \
                    labelMat[j] * (alphas[j] - alphaJold) * \
                    dataMatrix[i,:] * dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * \
                    dataMatrix[i,:] * dataMatrix[j,:].T - \
                    labelMat[j] * (alphas[j] - alphaJold) * \
                    dataMatrix[j,:] * dataMatrix[j,:].T
                
                if alphas[i] > 0 and alphas[i] < C:
                    b = b1
                elif alphas[j] > 0 and alphas[j] < C:
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                
                alphaPairsChanged += 1
                
        if alphaPairsChanged == 0:
            iter += 1
        else:
            iter = 0
        #print('iteration number: %d'%iter)
    return b,alphas
            

In [7]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)

In [8]:
b

matrix([[-3.80854664]])

In [9]:
alphas[alphas>0]

matrix([[4.04250855e-02, 3.18466179e-01, 1.09613813e-01, 2.49277452e-01,
         1.38777878e-17]])

In [10]:
for i in range(100):
    if alphas[i] > 0.0:
        print (dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0
[6.543888, 0.433164] 1.0


In [11]:
#6.4 SMO算法的加速优化

In [39]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2)))  #第1列表示是否有效的标志位，第2列表示实际的E值
        
def calcEk(oS,k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T * \
               (oS.X * oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i,oS,Ei):        #内循环的启发式方法
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    
    oS.eCache[i] = [1,Ei]
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:
            if k ==i:
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK,Ej
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]
    

In [26]:
##SMO算法中寻找决策边界的优化历程

In [27]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i] * Ei < -oS.tol and oS.alphas[i] < oS.C) or \
        (oS.labelMat[i] * Ei > oS.tol and oS.alphas[i] > 0):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
            
        if L == H:
            #print('L == H')
            return 0
        eta = 2.0 * oS.X[i,:] * oS.X[j,:].T - oS.X[i,:] * oS.X[i,:].T - \
              oS.X[j,:] * oS.X[j,:].T
        if eta >= 0:
           # print('eta >= 0')
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            #print('j not moving enough')
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS,i)
        
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
            oS.X[i,:] * oS.X[i,:].T - \
            oS.labelMat[j] * (oS.alphas[j] - alphaJold) * \
            oS.X[i,:] * oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * \
            oS.X[i,:] * oS.X[j,:].T - \
            oS.labelMat[j] * (oS.alphas[j] - alphaJold) * \
            oS.X[j,:] * oS.X[j,:].T
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[i]:
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

In [28]:
##完整的SMO外循环代码

In [29]:
def smoP(dataMatIn,classLabels, C, toler, maxIter, kTup=('lin',0)):
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    
    entireSet = True; alphaPairsChanged = 0
    while iter <maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:
            nonBounds = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBounds:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
            print("iteration number: %d" % iter)
    return oS.b,oS.alphas    

In [40]:
b,alphas = smoP(dataArr,labelArr,0.6,0.001,40)

fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
fullSet, iter: 0 i:7, pairs changed 2
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
fullSet, iter: 0 i:10, pairs changed 2
fullSet, iter: 0 i:11, pairs changed 2
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
fullSet, iter: 0 i:22, pairs changed 3
fullSet, iter: 0 i:23, pairs changed 3
fullSet, iter: 0 i:24, pairs changed 3
fullSet, iter: 0 i:25, pairs change

C:\Users\ITS\AppData\Local\Temp/ipykernel_17300/1773793700.py:23: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  oS.eCache[i] = [1,Ei]
C:\Users\ITS\AppData\Local\Temp/ipykernel_17300/1773793700.py:43: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  oS.eCache[k] = [1,Ek]


In [41]:
def calcWs(alphas,dataArr,classLabels):
    X = np.mat(dataArr); labelMat = np.mat(classLabels).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [43]:
ws = calcWs(alphas,dataArr,labelArr)

In [44]:
ws

array([[ 0.65307162],
       [-0.17196128]])

In [45]:
dataMat = np.mat(dataArr)

In [47]:
#测试结果
dataMat[0] * np.mat(ws) + b

matrix([[-0.92555695]])

In [48]:
labelArr[0]

-1.0